In [1]:
from polygon import RESTClient
from typing import cast
from urllib3 import HTTPResponse
from config import POLYGON_API_KEY
import json
from datetime import datetime
import time
import pandas as pd

In [2]:
# client = RESTClient(POLYGON_API_KEY)

# aggs = cast(
#     HTTPResponse,
#     client.get_aggs(
#         "JPM",
#         1,
#         "day",
#         "2022-09-14",
#         "2022-09-15",
#         raw=True,
#     ),
# )

In [3]:
time.mktime(datetime(2022, 9, 15, 11, 0).timetuple())*1e9

1.663254e+18

In [4]:
client = RESTClient(POLYGON_API_KEY)

quotes_aggs = cast(
    HTTPResponse,
    client.list_quotes(
        ticker="JPM",
        timestamp_gte=datetime(2022, 9, 15, 8, 0),
        timestamp_lte=datetime(2022, 9, 16, 8, 1),
        limit=5000,
        raw=True,
    ),
)

trades_aggs = cast(
    HTTPResponse,
    client.list_trades(
        ticker="JPM",
        timestamp_gte=datetime(2022, 9, 15, 8, 0),
        timestamp_lte=datetime(2022, 9, 16, 8, 1),
        limit=5000,
        raw=True,
    ),
)

In [5]:
print(quotes_aggs.geturl())
print(quotes_aggs.status)
print(trades_aggs.geturl())
print(trades_aggs.status)
quotes_aggs_res = json.loads(quotes_aggs.data.decode("utf-8"))['results']
trades_aggs_res = json.loads(trades_aggs.data.decode("utf-8"))['results']
print(f"{len(quotes_aggs_res)} quotes")
print(f"{len(trades_aggs_res)} trades")

https://api.polygon.io/v3/quotes/JPM
200
https://api.polygon.io/v3/trades/JPM
200
5000 quotes
5000 trades


print(json.dumps(quotes_aggs_res[:2], indent=4))
print(json.dumps(quotes_aggs_res[-2:], indent=4))

print(json.dumps(trades_aggs_res[:2], indent=4))
print(json.dumps(trades_aggs_res[-2:], indent=4))

In [6]:
# https://stackoverflow.com/questions/15649942/how-to-convert-epoch-time-with-nanoseconds-to-human-readable

def nanos2dt(row, ts_col):
    dt = datetime.fromtimestamp(row[ts_col] / 1e9)
    return '{}.{:09.0f}'.format(dt.strftime('%Y-%m-%dT%H:%M:%S'), row[ts_col] % 1e9)

In [7]:
def list_col_len_gt1(row, list_col):
    return(len(row[list_col]) > 1)

In [8]:
quotes_df = pd.json_normalize(quotes_aggs_res)

In [9]:
# pt_dt - participant datetime
quotes_df['pt_dt'] = quotes_df.apply(lambda row: nanos2dt(row, 'participant_timestamp'), axis=1)
quotes_df['st_dt'] = quotes_df.apply(lambda row: nanos2dt(row, 'sip_timestamp'), axis=1)
# quotes_df

In [10]:
q_ask_price_pt_ts_df = quotes_df[['ask_price', 'participant_timestamp']]
q_ask_price_pt_ts_df = q_ask_price_pt_ts_df.groupby('participant_timestamp').ask_price.apply(set).reset_index()
q_ask_price_pt_ts_df[q_ask_price_pt_ts_df.apply(lambda row: list_col_len_gt1(row, 'ask_price'), axis=1)]

,participant_timestamp,ask_price
924,1663271966164717000,"{117.91, 117.92}"
2061,1663271985335085592,"{117.9, 117.91}"
2372,1663271994647067322,"{117.86, 117.87}"


In [11]:
trades_df = pd.json_normalize(trades_aggs_res)

In [12]:
# pt_dt - participant datetime
trades_df['pt_dt'] = trades_df.apply(lambda row: nanos2dt(row, 'participant_timestamp'), axis=1)
trades_df['st_dt'] = trades_df.apply(lambda row: nanos2dt(row, 'sip_timestamp'), axis=1)
# quotes_df

In [15]:
t_ask_price_pt_ts_df = trades_df[['price', 'participant_timestamp']]
t_ask_price_pt_ts_df = t_ask_price_pt_ts_df.groupby('participant_timestamp').price.apply(set).reset_index()
t_ask_price_pt_ts_df[t_ask_price_pt_ts_df.apply(lambda row: list_col_len_gt1(row, 'price'), axis=1)]

,participant_timestamp,price
85,1663271772870061824,"{118.105, 118.1}"
109,1663271774827749888,"{118.085, 118.08}"
114,1663271774828572672,"{118.085, 118.08}"
123,1663271774830794000,"{118.085, 118.08}"
218,1663271779901173000,"{117.995, 117.99}"
...,...,...
4399,1663328019322437000,"{116.98, 116.96, 116.95, 116.99, 117.0, 116.94..."
4403,1663328019322980000,"{117.01, 117.02}"
4404,1663328026407084000,"{117.01, 117.0}"
4415,1663328729941544327,"{117.05, 117.02}"
